In [1]:
from datasets import load_dataset
from transformers import pipeline
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm import tqdm
import evaluate
import gc

# Loading Dataset

The dataset can be found at https://huggingface.co/datasets/code-search-net/code_search_net

In [2]:
dataset = load_dataset("Fsoft-AIC/the-vault-function", split_set=["test"])

Here are the columns in the dataset. We will be using `original_docstring` for documentation and `original_string` for code.

In [4]:
test_df = dataset['test'].to_pandas()

In [5]:
test_df.columns

Index(['hexsha', 'repo', 'path', 'license', 'language', 'identifier',
       'return_type', 'original_string', 'original_docstring', 'docstring',
       'docstring_tokens', 'code', 'code_tokens', 'short_docstring',
       'short_docstring_tokens', 'comment', 'parameters', 'docstring_params'],
      dtype='object')

Getting a random sample from the test split to test our baseline model. We will increase the size once we finalize the experimental plan

In [6]:
dataset_sample = test_df.groupby('language', group_keys=False).sample(n=20)

# Testing the Alternative model

Here we define the system prompt for the Llama 4 model.

In [7]:
def prompt(language):
    return \
f'''You are a helpful agent designed to simplify code documentation for beginner programmers.
You will be provided with a block of {language} code and the existing doucmentation that accompanies it.
Simplify the given documentation, using the provided code as context, so that it is understandable
to beginner programmers. Output absolutely nothing else besides the simplified documentation.
Make sure to keep any documentation formatting codes present in the simplified documentation.
If you feel that the existing documentation is simple enough and meaning would be lost by simplifying
it further, feel free to keep the documentation as is. Here is the original documentation and code:'''

Creating the pipeline for the Llama 2 model using the HuggingFace transformers library. Modified from the example here: https://huggingface.co/docs/transformers/en/model_doc/llama2

In [8]:
pipe = pipeline(
    task="text-generation",
    model="meta-llama/Llama-4-Scout-17B-16E-Instruct",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

Fetching 50 files:   0%|          | 0/50 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/50 [00:00<?, ?it/s]

Device set to use cuda:0


Testing the pipeline. Modified from examples given here: https://huggingface.co/docs/transformers/main/en/chat_templating

In [9]:
message = [
    {"role": "system", "content": prompt(dataset['test'][0]['language'])},
    {"role": "user", "content": f"Documentation:\n{dataset['test'][0]['original_docstring']}\n\nCode:\n{dataset['test'][0]['original_string']}"}
]
print(f"Original Documentation:\n{dataset['test'][0]['original_docstring']}\n")
print(f"Code:\n{dataset['test'][0]['original_string']}\n")
output = pipe(message, pad_token_id=pipe.tokenizer.eos_token_id, max_new_tokens=2000)
print("Simplified Documentation:\n" + output[0]['generated_text'][-1]['content'])

Original Documentation:
Build the library mappings tables.

Code:
def build_mapping_tables(app):
    """Build the library mappings tables."""
    env = Environment(loader=FileSystemLoader(f"{DIR_PATH}"))
    template_file = env.get_template("table_template.j2")

    LIST_OF_MAP_DICTS = []
    for attr in dir(lib_mapper):
        if (attr.endswith("MAPPER_REVERSE") or attr.endswith("_MAPPER")) and not (
            attr.startswith("_") or attr.startswith("NETMIKO") or attr.startswith("MAIN")
        ):
            LIST_OF_MAP_DICTS.append(attr)

    for dict_name in LIST_OF_MAP_DICTS:
        lib_name = dict_name.split("_")[0]
        filename = f"{lib_name}_reverse" if "REVERSE" in dict_name else lib_name
        headers = ["NORMALIZED", lib_name] if "REVERSE" in dict_name else [lib_name, "NORMALIZED"]
        rendered_template = template_file.render(lib_names=headers, mappings=getattr(lib_mapper, dict_name))
        with open(f"{DIR_PATH}/netutils/lib_mapping/{filename}_table.rst", "w

Loading the evaluation model used for computing semantic similarity. Taken from example here: https://huggingface.co/tasks/sentence-similarity

In [10]:
eval_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

Running inference on the dataset sample

In [12]:
semantic_similarities = []
metrics = evaluate.combine(['rouge', 'meteor'])

for instance in tqdm(dataset_sample.itertuples()):
    message = [
        {"role": "system", "content": prompt(instance.language)},
        {"role": "user", "content": f"Documentation:\n{instance.original_docstring}\n\nCode:\n{instance.original_string}"}
    ]
    
    gc.collect()
    torch.cuda.empty_cache()

    result = pipe(message, pad_token_id=pipe.tokenizer.eos_token_id, max_new_tokens=2000)[0]['generated_text'][-1]['content']

    embedding_original = eval_model.encode(instance.original_docstring, convert_to_tensor=True)
    embedding_predicted = eval_model.encode(result, convert_to_tensor=True)

    semantic_similarities.append(util.pytorch_cos_sim(embedding_original, embedding_predicted).item())
    metrics.add(predictions=result, references=instance.original_docstring)

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/j/jwoods03/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/j/jwoods03/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/j/jwoods03/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
200it [2:01:46, 36.53s/it]


Summary statistics for semantic similarity results (alternative model)

In [13]:
sims = pd.DataFrame(semantic_similarities)
sims.to_excel('results/semantic_similarities_llama4.xlsx')
sims.describe()

,0
count,200.000000
mean,0.745733
std,0.177670
min,-0.066537
25%,0.654670
50%,0.793024
75%,0.874772
max,0.985788


ROUGE and METEOR results

In [14]:
import json
with open('results/rouge_meteor_llama4.json', 'w') as file:
    mr = metrics.compute()
    json.dump(mr, file, indent=4)